### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [223]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from datetime import datetime

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [35]:
n_movies = movies.shape[0]

In [4]:
movies['genre'].value_counts()

Drama                                                                      3602
Comedy                                                                     2091
Documentary                                                                1443
Comedy|Drama                                                               1371
Drama|Romance                                                              1199
Comedy|Drama|Romance                                                        988
Comedy|Romance                                                              846
Horror                                                                      822
Horror|Thriller                                                             563
Drama|Thriller                                                              503
Crime|Drama|Thriller                                                        480
Crime|Drama                                                                 434
Thriller                                

In [30]:
genres = movies['genre'].str.split('|', expand = True)
cols = range(0, 10)

In [158]:
genre_list = []
for col in cols:
    for element in genres[col].unique():
        if element:
            genre_list.append(element)
genre_list = list(set(genre_list))

In [161]:
try:
    genre_list.remove(np.nan)
except:
    print('No nan found')

In [37]:
n_genres = len(genre_list)

In [39]:
movies.isnull().mean()

movie_id    0.000000
movie       0.000000
genre       0.007018
dtype: float64

In [40]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [48]:
reviews.isnull().mean()

user_id      0.0
movie_id     0.0
rating       0.0
timestamp    0.0
dtype: float64

In [44]:
n_reviews = reviews.shape[0]

In [45]:
n_users = reviews['user_id'].nunique()

In [56]:
n_reviews/n_users

12.825946876902291

In [50]:
mean_rat = reviews['rating'].mean()

In [52]:
min_rat = reviews['rating'].min()

In [53]:
max_rat = reviews['rating'].max()

In [55]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': n_movies,
'The number of ratings in the dataset': n_reviews,
'The number of different genres': n_genres,
'The number of unique users in the dataset': n_users,
'The number missing ratings in the reviews dataset': reviews.isnull().mean()['rating'],
'The average rating given across all ratings': mean_rat,
'The minimum rating given across all ratings': min_rat,
'The maximum rating given across all ratings': max_rat
}

print(dict_sol1)


{'The number of movies in the dataset': 35479, 'The number of ratings in the dataset': 863866, 'The number of different genres': 29, 'The number of unique users in the dataset': 67353, 'The number missing ratings in the reviews dataset': 0.0, 'The average rating given across all ratings': 7.3158776939942074, 'The minimum rating given across all ratings': 0, 'The maximum rating given across all ratings': 10}


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [224]:
# Removing year parenthesis and renaming columns
clean_movie = movies['movie'].str.split("(", expand=True)
clean_movie[1] = clean_movie[1].str.replace(")", "")
clean_movie = clean_movie.rename(columns={0: "movie", 1: "year"})

In [225]:
# Droping old movie column and adding new ones (name and year)
new_movies = pd.concat([movies.drop(columns=['movie']), clean_movie], axis = 1)

In [226]:
# Getting first two digits of year and creating dummy columns with century
new_movies['cent'] = new_movies['year'].str[:2]
new_movies = pd.get_dummies(new_movies, columns=['cent'])
new_movies = new_movies.rename(columns={'cent_18': '1800s', 'cent_19': '1900s','cent_20': '2000s'})

In [227]:
# Replacing NaN Genres with "Empty"
new_movies.loc[(new_movies['genre'].isnull() == True), 'genre'] = 'Empty'

In [228]:
# Creating dummy columns for each movie genre
new_genres = pd.DataFrame()
for genre in genre_list:
    new_genres[genre] = new_movies['genre'].str.contains(genre).astype('int64')
final_movies = pd.concat([new_movies, new_genres], axis = 1)

In [229]:
final_movies.head()

,movie_id,genre,movie,year,1800s,1900s,2000s,Reality-TV,Western,Horror,...,Family,Mystery,Animation,History,Sport,War,Film-Noir,Adult,Music,Drama
0,0000008,Documentary|Short,Edison Kinetoscopic Record of a Sneeze,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,Documentary|Short,La sortie des usines Lumière,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,Documentary|Short,The Arrival of a Train,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,Empty,The Oxford and Cambridge University Boat Race,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Short|Horror,Le manoir du diable,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [230]:
# Reviews - checking timestamp column
reviews.dtypes

user_id      object
movie_id     object
rating        int64
timestamp    object
dtype: object

In [231]:
# Creating copy of reviews and converting timestamp to int
new_reviews = reviews.copy()
new_reviews['timestamp'] = new_reviews['timestamp'].astype('int64')

In [233]:
# Date Column
new_reviews['date'] = new_reviews['timestamp'].apply(datetime.fromtimestamp)

In [243]:
# Month and year columns - as str to help get_dummies later
new_reviews['month'] = new_reviews['date'].apply(lambda x: x.month).astype('str')
new_reviews['year'] = new_reviews['date'].apply(lambda x: x.year).astype('str')

In [251]:
# Dummies for month and year
new_reviews = pd.get_dummies(new_reviews, columns = ['month', 'year'])

In [252]:
new_reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_10,month_11,month_12,month_2,...,month_8,month_9,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
0,1,0114508,8,1381006850,2013-10-05 21:00:50,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2,0208092,5,1586466072,2020-04-09 21:01:12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0358273,9,1579057827,2020-01-15 03:10:27,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,10039344,5,1578603053,2020-01-09 20:50:53,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,6751668,9,1578955697,2020-01-13 22:48:17,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [82]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year